# Implementation of Blackjack card game.

Did it just to understand what are the rules.

In [258]:
from collections import namedtuple
import random
from typing import List
import itertools

In [69]:
TWENTY_ONE = 21
VALUES = {
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10": 10,
    "J": 10,
    "Q": 10,
    "K": 10,
    "A": 11  # could be one if the hand exceeds 21
}
CardTuple = namedtuple("Card", "rank suit")
class Card(CardTuple):
    """Represents a single card."""
    @property
    def value(self):
        return VALUES[self.rank]

In [41]:
c = Card("4", "S")

**Suits**:
* Spade ♠
* Heart ♥
* Diamond ♦
* Club ♣

**Ranks**:

2, 3, 4, 5, 6, 7, 8, 9, 10, J, Q, K, A

In [3]:
a = Card("A", "S")

In [4]:
a

Card(rank='A', suit='S')

In [9]:
SUITS = ("S", "H", "D", "C")
RANKS = tuple(str(i) for i in range (2, 11)) + tuple("J Q K A".split())
RANKS

('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')

In [12]:
deck = [Card(rank, suit) for rank in RANKS for suit in SUITS]
deck[:5]

[Card(rank='2', suit='S'),
 Card(rank='2', suit='H'),
 Card(rank='2', suit='D'),
 Card(rank='2', suit='C'),
 Card(rank='3', suit='S')]

In [39]:
random.shuffle(deck)
deck[:5]

[Card(rank='7', suit='S'),
 Card(rank='7', suit='H'),
 Card(rank='4', suit='H'),
 Card(rank='A', suit='C'),
 Card(rank='3', suit='D')]

In [20]:
"""Class representing a game of single player against a dealer"""
class Dealer:
    def __init__(self):
        self.deck = [Card(rank, suit) for rank in RANKS for suit in SUITS]
        random.shuffle(self.deck)
        
    def get_card(self) -> Card:
        return self.deck.pop()

In [188]:
class ActionNotAllowed(Exception):
    pass

In [386]:
class Hand:
    "Class representing a hand (a list of cards). One player can have one or more hands (after splitting)"
    def __init__(self, cards: List[Card]):
        self.hand = cards
        self.stand_commitment = False
        self.active = True
        self.value  # calculates soft_hand property
        
    def __str__(self):
        return str(self.hand)
    
    def __repr__(self):
        return repr(self.hand)
    
    def __len__(self):
        return len(self.hand)
    
    def __getitem__(self, i):
        return self.hand[i]
    
    def append(self, item: Card):
        self.hand.append(item)
        
    def pop(self):
        return self.hand.pop()
    
    @property
    def value(self) -> int:
        n_aces = 0
        current_value = 0
        for card in self.hand:
            current_value += VALUES[card.rank]
            if card.rank == "A":
                n_aces += 1
        while n_aces > 0 and current_value > TWENTY_ONE:
            current_value -= 10
            n_aces -= 1
        if n_aces > 0 and current_value <= TWENTY_ONE:  # hand could be soft also when it's value equals 21
            self.soft_hand = True
        else:
            self.soft_hand = False
        return current_value
    
    def bust(self) -> bool:
        return self.hand_value > TWENTY_ONE    

In [412]:
class Player:
    def __init__(self):
        self.dealer = Dealer()
        # State of the player.
        self.n_wins = 0
        self.n_plays = 0
        # Hands (assuming casino allows us to play maximum of 4 hands).
        self.first_hand = Hand([self.dealer.get_card(), self.dealer.get_card()])
        self.second_hand = Hand([])
        self.third_hand = Hand([])
        self.fourth_hand = Hand([])
        self.hands = [self.first_hand, self.second_hand, self.third_hand, self.fourth_hand]
        self.current_hand = 0
        self.n_hands = 1
        
    def __str__(self):
        return "Player(\n \t n_hands={},\n\t".format(self.n_hands) + ",\n\t".join(str(hand) for hand in self.hands[:self.n_hands]) + "\n\t)"
    
    def __repr__(self):
        return str(self)
        
    @property
    def hand(self):
        return self.hands[self.current_hand]
    
    # Actions        
    def show_hand(self):
        print("Hand(n={}, {})".format(self.current_hand, self.hand))
    
    def hit(self):
        if self.hand.stand_commitment:
            raise ActionNotAllowed
        self.hand.append(self.dealer.get_card())  # adding a card from the dealer
        self.hand.value  # in order to evaluate if the hand is soft or not
        
    def stand(self):
        pass  # nothing to be done
    
    def double_down(self):
        self.hand.hit()
        self.hand.stand_commitment = True
    
    def split(self):
        if len(self.hand) != 2 or self.hand[0].value != self.hand[1].value or self.n_hands >= 4:  # number of dealing hands can't exceed 4
            raise ActionNotAllowed("Cannot split this hand.")
        self.hands[self.n_hands].append(self.hand.pop()) # popping one item of the current hand into last one
        self.n_hands += 1
        
    # Done after taking some of the four actions
    def change_hand(self):
        self.current_hand = (self.current_hand + 1) % self.n_hands
       

In [480]:
player = Player()

In [481]:
player

Player(
 	 n_hands=1,
	[Card(rank='5', suit='S'), Card(rank='5', suit='C')]
	)

In [482]:
player.show_hand()

Hand(n=0, [Card(rank='5', suit='S'), Card(rank='5', suit='C')])


In [483]:
player.split()

In [484]:
player.hand.value

5

In [486]:
player

Player(
 	 n_hands=2,
	[Card(rank='5', suit='S')],
	[Card(rank='5', suit='C')]
	)

In [487]:
player.hit()

In [488]:
player

Player(
 	 n_hands=2,
	[Card(rank='5', suit='S'), Card(rank='2', suit='C')],
	[Card(rank='5', suit='C')]
	)